# Replication code for "The impacts of the Amazon Soy Moratorium on deforestation" 
Heilmayr, Rausch, Munger and Gibbs

## Import packages and define data location

In [1]:
import sys
sys.path.append('D:/dev/glue-sb/')
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
%matplotlib inline 
import numpy as np
import dirfuncs
import plot_tools

In [2]:
dropbox_dir = dirfuncs.guess_dropbox_dir()
data_dir = dropbox_dir + 'soyM/analysis/7-21-20/'

## Set plotting parameters

In [3]:
palette = ['#CD6699', '#448970', '#F5CA7A']
palette = {'notleg': palette[2],
           'notbiom': palette[0],
           'soym': palette[1],
           'neutral': 'black'}
fig_dir = data_dir + 'figures/'
font = {'family' : 'sans-serif',
        'weight': 'normal',
        'size'   : 16}
matplotlib.rc('font', **font)

## Load and prep data

In [ ]:
defor_csv = data_dir + 'long.csv'
defor_df = pd.read_csv(defor_csv)
# defor_df['on_car'] = np.logical_not(defor_df['propid'].isnull())
# defor_df['gts_car'] = defor_df['gts'].astype(int) & defor_df['on_car']
defor_df['late'] = defor_df['year']>2005

In [ ]:
buffer = 300
sample_df = defor_df.loc[((defor_df['dist_amb']>-300) & (defor_df['dist_aml']<300))]

In [ ]:
small_buffer = 100
small_sample_df = defor_df.loc[((defor_df['dist_amb']>-small_buffer) & (defor_df['dist_amb']<small_buffer))]

In [ ]:
wide_csv = data_dir + 'wide.csv'
wide_df = pd.read_csv(wide_csv)

In [ ]:
sample_df.loc[:,'gaez_soy_suit'] = (sample_df['GAEZsuit']>40).astype(int)
sample_df.loc[:,'soy_suit'] = ((sample_df['gaez_soy_suit']==1) & (sample_df['suit']>0)).astype(int)
small_sample_df.loc[:,'gaez_soy_suit'] = (small_sample_df['GAEZsuit']>40).astype(int)
small_sample_df.loc[:,'soy_suit'] = ((small_sample_df['gaez_soy_suit']==1) & (small_sample_df['suit']>0)).astype(int)
wide_df.loc[:,'gaez_soy_suit'] = (wide_df['GAEZsuit']>40).astype(int)
wide_df.loc[:,'soy_suit'] = ((wide_df['gaez_soy_suit']==1) & (wide_df['suit']>0)).astype(int)
suit_var = 'soy_suit'

In [ ]:
soy_csv = data_dir + 'soy_conversion.csv'
soy_df = pd.read_csv(soy_csv)

In [ ]:
# Regression results - generated by running regressions.do
t1_csv = data_dir + 'tables/t1_ddd.csv'
t2_csv = data_dir + 'tables/t2.csv'
tb_csv = data_dir + 'tables/tsb.csv'

## Figure 2 - Primary trend comparison

In [ ]:
out_var = 'mb2_vdefor'
defor = pd.pivot_table(sample_df, columns = 'year', index = [suit_var, 'biome', 'legal_amazon'], values = out_var, 
                       aggfunc = 'sum')
forest = pd.pivot_table(sample_df, columns = 'year', index = [suit_var, 'biome', 'legal_amazon'], values = out_var, 
                        aggfunc = 'count')
defor_shr = ((defor / forest) * 100).dropna(axis = 1).T.loc[2002:2016]
defor_shr = defor_shr.T

In [ ]:
def clean_time(time_csv):
    time_df = pd.read_csv(time_csv)
    time_df = time_df.iloc[1:-1]
    time_df.columns = ['var', 'estimate']
    time_df.loc[:,'var'] = np.repeat(time_df['var'].dropna(), 2).values
    time_df.loc[:,'stat'] = np.tile(['coef', 'ci'], int((time_df.shape[0]/2)))
    time_df = time_df.set_index(['stat', 'var'])
    time_df = time_df.unstack().T.reset_index()
    soy_rows = [var for var in time_df['var'] if (suit_var in var) & ('biome' in var) & ('year' in var)]
    df = time_df.loc[time_df['var'].isin(soy_rows)]
    df.loc[:,'year'] = df['var'].apply(lambda x: int(x[:4]))
    df.loc[:,'coef'] = df['coef'].apply(lambda x: float(x)) * 100
    df.loc[:,'lb'] = df['ci'].apply(lambda x: float(x.split(',')[0])) * 100
    df.loc[:,'ub'] = df['ci'].apply(lambda x: float(x.split(',')[1])) * 100
    df.loc[:,'err'] = df.loc[:,'coef'] - df.loc[:,'lb']
    return df


In [ ]:
time_csv = data_dir + 'figures/f2_time_plot.csv'
soytime_df = clean_time(time_csv)

In [ ]:
gs = gridspec.GridSpec(3, 1, height_ratios=[2, 2, 1.5]) 
fig = plt.figure(figsize=(8,12))
ax2 = plt.subplot(gs[2])
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharex = ax0, sharey = ax0)
ax0.plot(defor_shr.columns, defor_shr.loc[(0,0,0)], color = palette['notleg'], linestyle = '-')
ax0.plot(defor_shr.columns, defor_shr.loc[(0,0,1)], color = palette['notbiom'], linestyle = '-')
ax0.plot(defor_shr.columns, defor_shr.loc[(0,1,1)], color = palette['soym'], linestyle = '-')
ax1.plot(defor_shr.columns, defor_shr.loc[(1,0,0)], color = palette['notleg'], linestyle = '-')
ax1.plot(defor_shr.columns, defor_shr.loc[(1,0,1)], color = palette['notbiom'], linestyle = '-')
ax1.plot(defor_shr.columns, defor_shr.loc[(1,1,1)], color = palette['soym'], linestyle = '-')
sns.despine()
handles = [plt.Line2D(range(10), range(10), linestyle='-', color = palette['notleg'], linewidth = 2),
           plt.Line2D(range(10), range(10), linestyle='-', color = palette['notbiom'], linewidth = 2),
           plt.Line2D(range(10), range(10), linestyle='-', color = palette['soym'], linewidth = 2)]
ax0.set_ylabel('Deforestation rate outside\nsoy-suitable forests (%/y)')
ax1.set_ylabel('Deforestation rate within\nsoy-suitable forests (%/y)')
ax2.set_xlabel('Year')
ax0.axvline(x = 2006, color = 'grey', linestyle = '--')
ax1.axvline(x = 2006, color = 'grey', linestyle = '--')
labels = ['Cerrado biome, not legal Amazon', 'Cerrado biome, legal Amazon', 'Amazon biome, legal Amazon']
ax2.set_ylabel('Time varying treatment effect\n(percentage point)')
ax2.plot(soytime_df['year'].values, soytime_df['coef'].values, color = 'black')
ax2.fill_between(soytime_df['year'], soytime_df['lb'], soytime_df['ub'], alpha = 0.5, facecolor='grey', interpolate=False)
ax0.set_xticklabels([])
ax1.set_xticklabels([])
ax0.set_ylim((0,3.8))
ax2.axvline(x = 2006, color = 'grey', linestyle = '--')
ax2.axhline(y = 0, color = 'grey', linestyle = '--')
sns.despine()
ax0.annotate("A", xy = (2015, 3.5))
ax1.annotate("B", xy = (2015, 3.5))
ax2.annotate("C", xy = (2015, 0.6))
lgd = ax0.legend(handles, labels, fontsize = 10, frameon = False, loc='lower left', bbox_to_anchor=(0.5, 0.6))
lgd = ax1.legend(handles, labels, fontsize = 10, frameon = False, loc='lower left', bbox_to_anchor=(0.5, 0.6))
fig.tight_layout()
fig.savefig(data_dir + 'figures/f2_summary.svg')

# Figure S4 - Spatially bounded version of Figure 2

In [ ]:
out_var = 'mb2_vdefor'
small_sample_df.loc[:,'soy_suit'] = ((small_sample_df['gaez_soy_suit']==1) & (small_sample_df['suit']>0)).astype(int)
defor = pd.pivot_table(small_sample_df, columns = 'year', index = [suit_var, 'biome', 'legal_amazon'], values = out_var, 
                       aggfunc = 'sum')
forest = pd.pivot_table(small_sample_df, columns = 'year', index = [suit_var, 'biome', 'legal_amazon'], values = out_var, 
                        aggfunc = 'count')
defor_shr = ((defor / forest) * 100).dropna(axis = 1).T.loc[2002:2016]
defor_shr = defor_shr.T
time_csv = data_dir + 'figures/f2_time_plot_100.csv'
soytime_df = clean_time(time_csv)

In [ ]:
gs = gridspec.GridSpec(3, 1, height_ratios=[2, 2, 1.5]) 
fig = plt.figure(figsize=(8,12))
ax2 = plt.subplot(gs[2])
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharex = ax0, sharey = ax0)
ax0.plot(defor_shr.columns, defor_shr.loc[(0,0,1)], color = palette['notbiom'], linestyle = '-')
ax0.plot(defor_shr.columns, defor_shr.loc[(0,1,1)], color = palette['soym'], linestyle = '-')
ax1.plot(defor_shr.columns, defor_shr.loc[(1,0,1)], color = palette['notbiom'], linestyle = '-')
ax1.plot(defor_shr.columns, defor_shr.loc[(1,1,1)], color = palette['soym'], linestyle = '-')
sns.despine()
handles = [plt.Line2D(range(10), range(10), linestyle='-', color = palette['notbiom'], linewidth = 2),
           plt.Line2D(range(10), range(10), linestyle='-', color = palette['soym'], linewidth = 2)]
ax0.set_ylabel('Deforestation rate outside\nsoy-suitable forests (%/y)')
ax1.set_ylabel('Deforestation rate within\nsoy-suitable forests (%/y)')
ax2.set_xlabel('Year')
ax0.axvline(x = 2006, color = 'grey', linestyle = '--')
ax1.axvline(x = 2006, color = 'grey', linestyle = '--')
labels = ['Cerrado biome, legal Amazon', 'Amazon biome, legal Amazon']
ax2.set_ylabel('Time varying treatment effect\n(percentage point)')
ax2.plot(soytime_df['year'].values, soytime_df['coef'].values, color = 'black')
ax2.fill_between(soytime_df['year'], soytime_df['lb'], soytime_df['ub'], alpha = 0.5, facecolor='grey', interpolate=False)
ax0.set_xticklabels([])
ax1.set_xticklabels([])
ax0.set_ylim((0,5))
ax2.axvline(x = 2006, color = 'grey', linestyle = '--')
ax2.axhline(y = 0, color = 'grey', linestyle = '--')
sns.despine()
ax0.annotate("A", xy = (2015, 4.8))
ax1.annotate("B", xy = (2015, 4.8))
ax2.annotate("C", xy = (2015, 0.8))
lgd = ax0.legend(handles, labels, fontsize = 10, frameon = False, loc='lower left', bbox_to_anchor=(0.5, 0.6))
lgd = ax1.legend(handles, labels, fontsize = 10, frameon = False, loc='lower left', bbox_to_anchor=(0.5, 0.6))
fig.tight_layout()
fig.savefig(data_dir + 'figures/fs2_summary.svg')

## Figure 3 - Soy establishment trends

In [ ]:
soy_df = soy_df.loc[(soy_df['dist_amb']>-300) & (soy_df['dist_aml']<300)]
soy_subset = soy_df.loc[soy_df['a_start_soy']==0]

In [ ]:
soy_subset.loc[(soy_subset['year']==2017) & (soy_subset['biome']==1), 'mb_start_for'].value_counts()

In [ ]:
soy_subset.loc[(soy_subset['year']==2017) & (soy_subset['biome']==1) & (soy_subset['mb_start_for']==0), 'a_soy'].value_counts()

In [ ]:
summary_df = pd.pivot_table(soy_subset, index = 'a_soy', 
                            columns = ['year', 'mb_start_for', 'legal_amazon', 'biome'], values = 'ptid', aggfunc = len)
convert_df = (summary_df.loc[1] / (summary_df.loc[0] + summary_df.loc[1])).unstack(level=0)*100
convert_df = convert_df.rename(columns = {2006: '2000-2006', 2017: '2007-2017'})

In [ ]:
convert_df

In [ ]:
fig, ax = plt.subplots()
ax.plot(convert_df.loc[(0,0,0)], marker = 'o', linestyle = '--', color = palette['notleg'])
ax.plot(convert_df.loc[(0,1,0)], marker = 'o', linestyle = '--', color = palette['notbiom'])
ax.plot(convert_df.loc[(0,1,1)], marker = 'o', linestyle = '--', color = palette['soym'])
ax.plot(convert_df.loc[(1,0,0)], marker = 'o', linestyle = '-', color = palette['notleg'])
ax.plot(convert_df.loc[(1,1,0)], marker = 'o', linestyle = '-', color = palette['notbiom'])
ax.plot(convert_df.loc[(1,1,1)], marker = 'o', linestyle = '-', color = palette['soym'])
ax.set_ylabel('Percent of area\nconverted to soy')
ax.set_xlabel('Time period')
ax.set_xlim((-0.2, 1.2))

sns.despine()
handle_len = 50
handles = [plt.Line2D(range(handle_len), range(handle_len), linestyle='-', color = palette['notleg'], linewidth = 2),
           plt.Line2D(range(handle_len), range(handle_len), linestyle='-', color = palette['notbiom'], linewidth = 2),
           plt.Line2D(range(handle_len), range(handle_len), linestyle='-', color = palette['soym'], linewidth = 2)]
labels = ['Cerrado biome,\noutside legal Amazon', 'Cerrado biome,\nlegal Amazon', 'Amazon\nbiome']
lgd1 = ax.legend(handles, labels, fontsize = 8, frameon = False, loc = "upper left", bbox_to_anchor=(1.15,.9), handlelength = 4)
handles = [plt.Line2D(range(handle_len), range(handle_len), linestyle='--', color = palette['notleg'], linewidth = 2),
           plt.Line2D(range(handle_len), range(handle_len), linestyle='--', color = palette['notbiom'], linewidth = 2),
           plt.Line2D(range(handle_len), range(handle_len), linestyle='--', color = palette['soym'], linewidth = 2)]
labels = ['\n', '\n', '\n']
lgd2 = ax.legend(handles, labels, fontsize = 8, frameon = False, loc = "upper left", bbox_to_anchor=(1.,.9), handlelength=4)
ax.add_artist(lgd1)
ax.add_artist(lgd2)
ax.annotate('Non-forested\nlocations', xy = (1.075, 0.90), xycoords = "axes fraction", fontsize = 8, ha="center")
ax.annotate('Forested\nlocations', xy = (1.225, 0.90), xycoords = "axes fraction", fontsize = 8, ha="center")
fig.savefig(data_dir + 'figures/f3_conversion.svg', bbox_inches = 'tight')

## Create Figure S1 - Comparison to full Amazon

In [ ]:
defor = pd.pivot_table(sample_df, columns = 'year', index = ['biome', 'legal_amazon'], values = out_var, 
                       aggfunc = 'sum')
forest = pd.pivot_table(sample_df, columns = 'year', index = ['biome', 'legal_amazon'], values = out_var, 
                        aggfunc = 'count')
defor_shr = ((defor / forest) * 100).dropna(axis = 1).T.loc[2002:2016]
defor_shr = defor_shr.T
defor_all = pd.pivot_table(defor_df, columns = 'year', index = ['biome', 'legal_amazon'], values = out_var, 
                       aggfunc = 'sum')
forest_all = pd.pivot_table(defor_df, columns = 'year', index = ['biome', 'legal_amazon'], values = out_var, 
                        aggfunc = 'count')
defor_shr_all = ((defor_all / forest_all) * 100).dropna(axis = 1).T.loc[2002:2016]
defor_shr_all = defor_shr_all.T

In [ ]:
gs = gridspec.GridSpec(1, 2) 
fig = plt.figure(figsize=(15,4))
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1], sharex = ax0, sharey = ax0)
ax0.plot(defor_shr.columns, defor_shr_all.loc[(0, 0)], color = palette['notleg'], linestyle = '-')
ax0.plot(defor_shr.columns, defor_shr_all.loc[(0, 1)], color = palette['notbiom'], linestyle = '-')
ax0.plot(defor_shr.columns, defor_shr_all.loc[(1, 1)], color = palette['soym'], linestyle = '-')
ax1.plot(defor_shr.columns, defor_shr.loc[(0, 0)], color = palette['notleg'], linestyle = '-')
ax1.plot(defor_shr.columns, defor_shr.loc[(0, 1)], color = palette['notbiom'], linestyle = '-')
ax1.plot(defor_shr.columns, defor_shr.loc[(1, 1)], color = palette['soym'], linestyle = '-')
sns.despine()
handles = [plt.Line2D(range(10), range(10), linestyle='-', color = palette['notleg'], linewidth = 2),
           plt.Line2D(range(10), range(10), linestyle='-', color = palette['notbiom'], linewidth = 2),
           plt.Line2D(range(10), range(10), linestyle='-', color = palette['soym'], linewidth = 2)]
ax0.set_ylabel('Deforestation rate (%/y)')
ax0.set_xlabel('Year')
ax1.set_xlabel('Year')
ax0.axvline(x = 2006.5, color = 'grey', linestyle = '--')
ax1.axvline(x = 2006.5, color = 'grey', linestyle = '--')
labels = ['Cerrado biome, not legal Amazon', 'Cerrado biome, legal Amazon', 'Amazon biome, legal Amazon']
lgd = ax1.legend(handles, labels, fontsize = 8, frameon = False)
ax0.set_title("Full Amazon and Cerrado biomes")
ax1.set_title("Restricted Amazon and Cerrado biomes")
sns.despine()
fig.tight_layout()
ax0.xaxis.set_ticks(np.arange(2002, 2018, 2))
fig.savefig(data_dir + 'figures/methods_s1.svg')

## Paper calculations
### Impact calculations
Relative to the Cerrado portion of the Legal Amazon, the annual deforestation rate on soy-suitable locations declined by 0.68 +/- 0.38 pp in the Amazon biome after the adoption of the ASM (All error bounds describe the 95% confidence interval). Similarly, post-ASM deforestation in soy suitable regions of the Amazon biome declined by 0.99 +/- 0.24 pp relative to non-soy suitable portions of the biome. Using a triple differences model that integrates both of these comparisons, we estimate that the ASM reduced annual deforestation by 0.64 +/- 0.32 pp. This smaller estimated effect reflects the fact that soy-suitable areas of the Cerrado biome also experienced a relative decline (0.42 +/- 0.23 pp) in deforestation in the latter half of our study period. To quantify what would have happened had the ASM never been adopted, we construct a counterfactual scenario in which we add our estimated treatment effect (0.64 +/- 0.32 pp) to historical 2007-2016 deforestation rates. When compared to this counterfactual, the ASM reduced deforestation rates by 34% +/- 16, contributing 18 +/- 9 thousand km2 of avoided deforestation in the Amazon biome. 

In [ ]:
# Load regression table
t1 = pd.read_csv(t1_csv)
t1 = t1.loc[1:7]
t1.columns = ['drop', 'c1', 'c2', 'c3', 'c4', 'c5']
labels = ['soy_post_coef', 'soy_post_se', 'biome_post_coef', 'biome_post_se', 'ddd_coef', 'ddd_se', 'n']
t1['var'] = labels 
t1 = t1.drop('drop', axis = 1)
t1 = t1.replace(r'["=() ]','', regex = True)
t1

In [ ]:
coef = t1.loc[t1['var']=='biome_post_coef', 'c3'].astype(float).values[0]
se = t1.loc[t1['var']=='biome_post_se', 'c3'].astype(float).values[0].astype(float)
ci = 1.96 * se
print(coef, ci)

In [ ]:
coef = t1.loc[t1['var']=='soy_post_coef', 'c1'].astype(float).values[0]
se = t1.loc[t1['var']=='soy_post_se', 'c1'].astype(float).values[0].astype(float)
ci = 1.96 * se
print(coef, ci)

In [ ]:
att = t1.loc[t1['var']=='ddd_coef', 'c5'].astype(float).values[0]
att_se = t1.loc[t1['var']=='ddd_se', 'c5'].astype(float).values[0].astype(float)
att_ci = 1.96 * att_se
print(att, att_ci)

In [ ]:
coef = t1.loc[t1['var']=='soy_post_coef', 'c2'].astype(float).values[0]
se = t1.loc[t1['var']=='soy_post_se', 'c2'].astype(float).values[0].astype(float)
ci = 1.96 * se
print(coef, ci)

In [ ]:
out_var = 'mb2_vdefor'
defor = pd.pivot_table(sample_df, columns = 'year', index = ['soy_suit', 'biome', 'legal_amazon'], values = out_var, 
                       aggfunc = 'sum')
start_forest = pd.pivot_table(sample_df, columns = 'year', index = ['soy_suit', 'biome', 'legal_amazon'], values = out_var, 
                        aggfunc = 'count')
defor_shr = ((defor / start_forest) * 100).dropna(axis = 1).T.loc[2002:2016]
defor_shr = defor_shr.T

In [ ]:
bl_defor = (100-defor_shr.loc[(1,1,1), 2006:])/100
bl_cumdefor = bl_defor.cumprod().loc[2016]
start_for_05 = start_forest.loc[(1,1,1), 2006]
bl_forests = start_for_05 * bl_cumdefor
total_defor = (start_forest.loc[(1,1,1), 2006] - start_forest.loc[(1,1,1), 2017]) * 4

att_draw = np.random.normal(att, att_se, 10000)
cf_forests = pd.Series(map(lambda att: start_for_05 * ((att + bl_defor).cumprod().loc[2016]), att_draw))
avoided_defor = (bl_forests - cf_forests) * 4
mean = np.mean(avoided_defor)
std = np.std(avoided_defor)
ci = std * 1.96

In [ ]:
print("ASM saved " + str(int(mean)) + ' +/- ' + str(ci) + ' km2 of forests')
pct_reduction = mean / (total_defor + mean)
pct_reduction_ci = ci / (total_defor + mean)
print("ASM reduced total deforestation by " + str(int(100*pct_reduction)) + ' +/- ' + str(int(100*pct_reduction_ci)) + " percent")

In [ ]:
out_var = 'mb2_vdefor'
defor = pd.pivot_table(sample_df, columns = 'year', index = ['biome', 'legal_amazon'], values = out_var, 
                       aggfunc = 'sum')
start_forest = pd.pivot_table(sample_df, columns = 'year', index = ['biome', 'legal_amazon'], values = out_var, 
                        aggfunc = 'count')
defor_shr = ((defor / start_forest) * 100).dropna(axis = 1).T.loc[2002:2016]
defor_shr = defor_shr.T
defor_shr.loc[1,1]

### Complementarities with public policies
In addition, our estimate of the impact of the ASM represents only 25 percent of the 2.6 pp reduction in deforestation rates that occurred between 2002 and 2016 on soy-suitable locations in the Amazon biome portion of the Arc of Deforestation.

In [ ]:
out_var = 'mb2_vdefor'
defor = pd.pivot_table(sample_df, columns = 'year', index = [suit_var, 'biome', 'legal_amazon'], values = out_var, 
                       aggfunc = 'sum')
forest = pd.pivot_table(sample_df, columns = 'year', index = [suit_var, 'biome', 'legal_amazon'], values = out_var, 
                        aggfunc = 'count')
defor_shr = ((defor / forest) * 100).dropna(axis = 1).T.loc[2002:2016]
defor_shr = defor_shr.T

In [ ]:
reduction = (defor_shr.loc[(1,1,1)][2003] - defor_shr.loc[(1,1,1)][2016])
print(reduction)

In [ ]:
att * -100 / reduction

In [ ]:
# Load regression table
tb = pd.read_csv(tb_csv)
tb = tb.loc[0:15]
tb.columns = ['drop', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']
variables = ['a', 'a', 'b', 'b', 'c', 'c', 'd', 'd', 'e', 'e', 'f', 'f', 'leg_am', 'leg_am', 'g', 'g']
tb['var'] = variables 
stats = ['coef', 'se'] * 8
tb['stat'] = stats
tb = tb.drop('drop', axis = 1)
tb = tb.replace(r'["=() ]','', regex = True)
coef = tb.loc[(tb['var']=='leg_am') & (tb['stat']=='coef'), 'c6'].astype(float).values[0]
se = tb.loc[(tb['var']=='leg_am') & (tb['stat']=='se'), 'c6'].astype(float).values[0]
ci = 1.96 * se
print(coef, ci)

In [ ]:
# Load regression table
t2 = pd.read_csv(t2_csv)
t2 = t2.replace(r'["=() ]','', regex = True)
t2 = t2.loc[2:11]
t2.columns = ['var', 'c1', 'c2', 'c3', 'c4']

t2

### Research design
Since 90 percent of soy is planted in locations that meet specific soil and climatic suitability conditions, 

In [ ]:
subset = wide_df.loc[(wide_df['dist_amb']>-buffer) & (wide_df['dist_aml']<buffer)]
suit_df = subset.pivot_table(index = 'a_soy_2017', columns = suit_var, values = 'ptid', aggfunc = len).loc[1]
suit_df[[1,2]].sum() / suit_df.sum()

### Avoided deforestation
Prior to the adoption of the ASM, deforestation rates were similar on soy-suitable lands across the Amazon biome (XX% per year), the Cerrado-portion of the Legal Amazon (XX% per year) as well as the portion of the Cerrado biome outside of the Legal Amazon (XX% per year). Between 2006 and 2016, soy-suitable deforestation rates in the Amazon biome fell to XX%/year, XX pp/year below the rate of soy-suitable deforestation in the Cerrado portion of the Legal Amazon and XX pp/year below the rate in the Cerrado biome outside of the legal Amazon. 

In [ ]:
defor = pd.pivot_table(sample_df.loc[sample_df['soy_suit']==1], columns = 'year', index = ['biome', 'legal_amazon'], values = 'mb2_vdefor', 
                       aggfunc = 'sum')
forest = pd.pivot_table(sample_df.loc[sample_df['soy_suit']==1], columns = 'year', index = ['biome', 'legal_amazon'], values = 'mb2_vdefor', 
                        aggfunc = 'count')
defor_shr = ((defor / forest) * 100).dropna(axis = 1).T.loc[2002:2017]
defor_shr = defor_shr.T
defor_shr[list(range(2002, 2006))].mean(axis = 1)

In [ ]:
p2_rates = defor_shr[list(range(2006, 2017))].mean(axis = 1)
p2_rates

In [ ]:
p2_rates.loc[(0,1)] - p2_rates.loc[(1,1)]

In [ ]:
p2_rates.loc[(0,0)] - p2_rates.loc[(1,1)]

## Methods calculations
### Study region
This region captures 96 percent of the 2017 soy area in the Amazon biome as well as 96 percent of the area of forests converted to soy between 2000 and 2017. In addition, our study region contains almost all (94 percent) of the forests monitored by the Soy Working Group (GTS) between 2007 and 2014. 

In [ ]:
defor_df['arc'] = (defor_df['dist_amb']>-300) & (defor_df['dist_aml']<300)

In [ ]:
defor = pd.pivot_table(defor_df.loc[defor_df['biome']==1], columns = 'year', 
                       index = 'arc', values = 'a_soy_2017', aggfunc = 'sum')
defor = defor.sum(axis = 1)
defor[True] / defor.sum()

In [ ]:
soy_defor = pd.pivot_table(defor_df.loc[(defor_df['biome']==1) & (defor_df['a_soy_2017']==1)], columns = 'year', 
                       index = 'arc', values = 'mb2_vdefor', aggfunc = 'sum')
soy_defor = soy_defor.sum(axis = 1)
soy_defor[True] / soy_defor.sum()

In [ ]:
subset = defor_df.loc[(defor_df['mb2_vfor_2000']==1) & (defor_df['year']<2015)]
ever_gts = subset.pivot_table(index = 'ptid', values = 'gts', aggfunc = max)
ever_arc = subset.pivot_table(index = 'ptid', values = 'arc', aggfunc = max)
ever_gts = ever_gts.merge(ever_arc, left_index = True, right_index = True, how = 'left')
ctab = pd.crosstab(ever_gts['gts'], ever_gts['arc'])
ctab.loc[1, True] / ctab.loc[1].sum()

### Sample creation
Across our study region, we sampled observations at each vertex of a grid of evenly spaced (2km) horizontal and vertical lines. This produced XX sample points with XX of those points falling inside the Amazon biome and XX points falling inside the Cerrado biome.  For each of these points, we extracted data from a variety of sources as outlined below.

In [ ]:
buffer = 300
subset = wide_df.loc[(wide_df['dist_amb']>-buffer) & (wide_df['dist_aml']<buffer)]
pt_description = subset.pivot_table(index = 'biome', values = 'ptid', aggfunc = len)

In [ ]:
pt_description

In [ ]:
pt_description.sum()

In [ ]:
# Show that this generates same forest sample as primary model specification
subset = wide_df.loc[(wide_df['dist_amb']>-buffer) & (wide_df['dist_aml']<buffer) & (wide_df['legal_amazon']==1) & ((wide_df['mb2_y_defor']>=2002) | (pd.isnull(wide_df['mb2_y_defor'])))]
pt_description = subset.pivot_table(index = 'biome', columns = 'mb2_vfor_2000', values = 'ptid', aggfunc = len)
pt_description[1].sum()

### Functional form
However, given that annual deforestation events are relatively rare (mean annual deforestation = XX), a non-linear functional form might better represent our binary outcome variable.

In [ ]:
out_var = 'mb2_vdefor'
defor = pd.pivot_table(sample_df, columns = 'year', index = ['legal_amazon'], values = out_var, 
                       aggfunc = 'sum')
forest = pd.pivot_table(sample_df, columns = 'year', index = ['legal_amazon'], values = out_var, 
                        aggfunc = 'count')
defor_shr = ((defor / forest) * 100).dropna(axis = 1).T.loc[2002:2016]
defor_shr[1].mean()

### Point and property fixed effects
To estimate this model we restricted our sample to the XX percent of points that fell within a property listed in one of the state or federal registries. 

In [ ]:
subset = wide_df.loc[(wide_df['dist_amb']>-buffer) & (wide_df['dist_aml']<buffer) & (wide_df['legal_amazon']==1)]
(subset['propid'].notnull()).astype(int).describe()['mean']